# TF-IDF PARA LA DESCRICPCIÓN

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_parquet('./csv/complete.parquet')
df.head()

,backdrop_path,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_count,cast,keywords,director,year,director_dict,cast_dict,description,poster_id,backdrop_id
8071,https://image.tmdb.org/t/p/w500//gMJngTNfaqCSC...,200000000,"[Action, Adventure, Science Fiction]",640146,tt10954600,en,Ant-Man and the Wasp: Quantumania,Super-Hero partners Scott Lang and Hope van Dy...,8567.865,https://image.tmdb.org/t/p/w500//ngl2FKBlU4fhb...,...,1891,"[Owen Wilson, Kathryn Newton, Paul Rudd, Evang...","[hero, ant, sequel, superhero, based on com, f...",[Peyton Reed],2023,"[{'adult': False, 'credit_id': '5dbc9866d388ae...","[{'adult': False, 'cast_id': 188, 'character':...",Super-Hero partners Scott Lang and Hope van Dy...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/gMJngTNfaqCSCqGD4y8lVMZXKDn.jpg
8072,https://image.tmdb.org/t/p/w500//iJQIbOPm81fPE...,100000000,"[Animation, Adventure, Family, Fantasy, Comedy]",502356,tt6718170,en,The Super Mario Bros. Movie,"While working underground to fix a water main,...",6572.614,https://image.tmdb.org/t/p/w500//qNBAXBIQlnOTh...,...,1462,"[Jack Black, Chris Pratt, Anya Taylor-Joy, Cha...","[video gam, plumber, magic mushroom, based on ...","[Michael Jelenic, Aaron Horvath]",2023,"[{'adult': False, 'credit_id': '61260ba84e6742...","[{'adult': False, 'cast_id': 14, 'character': ...","While working underground to fix a water main,...",/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iJQIbOPm81fPEGKt5BPuZmfnA54.jpg
8073,https://image.tmdb.org/t/p/w500//wybmSmviUXxlB...,125000000,"[Action, Comedy, Fantasy]",594767,tt10151854,en,Shazam! Fury of the Gods,"Billy Batson and his foster siblings, who tran...",4274.232,https://image.tmdb.org/t/p/w500//2VK4d3mqqTc7L...,...,1233,"[Grace Caroline Currey, Rachel Zegler, Gal Gad...","[superhero, end of the world, super pow, after...",[David F. Sandberg],2023,"[{'adult': False, 'credit_id': '5cb23bab0e0a26...","[{'adult': False, 'cast_id': 10, 'character': ...","Billy Batson and his foster siblings, who tran...",/2VK4d3mqqTc7LVZLnLPeRiPaJ71.jpg,/wybmSmviUXxlBmX44gtpow5Y9TB.jpg
8074,https://image.tmdb.org/t/p/w500//ovM06PdF3M8wv...,350000000,"[Science Fiction, Adventure, Action]",76600,tt1630029,en,Avatar: The Way of Water,Set more than a decade after the events of the...,3365.913,https://image.tmdb.org/t/p/w500//t6HIqrRAclMCA...,...,7544,"[Sam Worthington, Zoe Saldaña, Jack Champion, ...","[loss of loved on, dying and death, alien life...",[James Cameron],2022,"[{'adult': False, 'credit_id': '52fe4943c3a368...","[{'adult': False, 'cast_id': 209, 'character':...",Set more than a decade after the events of the...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/ovM06PdF3M8wvKb06i4sjW3xoww.jpg
8075,https://image.tmdb.org/t/p/w500//xwA90BwZXTh8k...,0,"[Action, Adventure, History, Drama, War]",948713,tt15767808,en,The Last Kingdom: Seven Kings Must Die,"In the wake of King Edward's death, Uhtred of ...",3119.049,https://image.tmdb.org/t/p/w500//7yyFEsuaLGTPu...,...,232,"[Alexander Dreymon, Emily Cox, Arnas Fedaravic...","[fight, kingdom, battle for pow, heir to the t...",[Edward Bazalgette],2023,"[{'adult': False, 'credit_id': '622cea60ea89f5...","[{'adult': False, 'cast_id': 8, 'character': '...","In the wake of King Edward's death, Uhtred of ...",/7yyFEsuaLGTPul5UkHc5BhXnQ0k.jpg,/xwA90BwZXTh8ke3CVsQlj8EOkFr.jpg


In [2]:
df['year'] = df['year'].astype('int')
df['keywords'] = df['keywords'].apply(lambda x: x if x != None else [])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


descriptions = df['description'].replace(np.nan, '')

# Crea una instancia del vectorizador TF-IDF
tfidf = TfidfVectorizer()
# Transforma las descripciones en una matriz de términos y documentos (DTM)
dtm = tfidf.fit_transform(descriptions)
# Calcula la similitud coseno entre todas las películas
similarity_matrix = cosine_similarity(dtm)
similarity_matrix[:10]


In [ ]:
sim_matrix = pd.DataFrame(similarity_matrix)

In [ ]:
sim_matrix.head()

# PRUEBA RECOMENDADOR

In [14]:
def compararString(str1, str2):
    if str1 == str2: return 10
    else: return 0

def compararListas(lista1, lista2):
    try:
        if len(lista1)== 0 or len(lista2) == 0:
            return 0
        set1 = set(lista1)
        set2 = set(lista2)
        if set1.issubset(set2):
            return 10
        else:
            intersection = set1.intersection(set2)
            union = set1.union(set2)
            jaccard_similarity = len(intersection) / len(union)
            score = jaccard_similarity * 10
        return score
    except:
        return 0


def similitud_anios(anio1, anio2, k):
    # Calcular la diferencia de años entre los dos valores
    diferencia = abs(anio1 - anio2)
    # Calcular la similitud utilizando la fórmula exponencial
    similitud = math.exp(-k * diferencia)
    return similitud

def sacar_puntuaciones_completo(df, kw_list, genero, actores, director, año, sim_dict):
    results_list = []
    i = 0
    for index, row in df.iterrows():
        keywords_p = compararListas(kw_list, row["keywords"])  # Sobre 10
        genres_p = compararListas(genero, row["genres"])  # Sobre 10
        cast_p = compararListas(actores, row["cast"])  # Sobre 10
        years_p = similitud_anios(int(año), int(row["year"]), 0.11)  # Sobre 1
        director_p = compararListas(director, row["director"])  # Sobre 10
        description_p = sim_dict[i] # sobre 1
        # kw * 0.3; genres * 0.2; cast * 0.05; years * 0.5; director * 0.2; description * 2

        labels = [keywords_p, genres_p, cast_p, years_p, director_p, description_p]
        values = [0.3, 0.20, 0.05, 0.5, 0.2, 2]
        points = sum(list(map(lambda x, y: x * y, labels, values)))
        results = {
            "keywords": keywords_p,
            "genres": genres_p,
            "cast": cast_p,
            "years": years_p * 10,
            "director": director_p,
            "description": description_p * 10,
        }
        results_list.append((row["id"], points, results))
        i += 1
    return results_list

def process_reccom(df, movie_id, similarity_matrix):
    # sacar_puntuaciones_completo(df, kw_list, genero, actores, director, año, sim_dict)
    movies = df[df["id"] == 502356]
    for index, row in movies.iterrows():
        movie = row.to_dict()

    sim_dict = description_similarity_dic(movie_id, similarity_matrix, list(df["id"]))
    list_res = sacar_puntuaciones_completo(
        df,
        movie["keywords"],
        movie["genres"],
        movie["cast"],
        movie["director"],
        movie["year"],
        sim_dict,
    )
    list_res.sort(key=lambda x: x[1], reverse=True)
    return list_res


def get_top_recommended_movies(df: pd.DataFrame, recommendations_list: list, limit: int):
    def search_points(rec_list, id, limit):
        for i in range(limit):
            m = rec_list[i]
            if m[0] == id:
                return (recommendations_list[i][1], recommendations_list[i][2])
        return None, None

    filtered_results = [x[0] for x in recommendations_list][:limit]
    filtered_df = df[df["id"].isin(filtered_results)]
    movies = []
    for idx, row in filtered_df.iterrows():
        movie = row.to_dict()
        points, results = search_points(recommendations_list, movie['id'], limit)
        movie["points"] = points
        movie['results'] = results
        movies.append(movie)
    return movies

In [15]:

movies = df[df['id'] == 502356]
for index, row in movies.iterrows():
        movie = row.to_dict()

movie

{'backdrop_path': 'https://image.tmdb.org/t/p/w500//iJQIbOPm81fPEGKt5BPuZmfnA54.jpg',
 'budget': 100000000,
 'genres': array(['Animation', 'Adventure', 'Family', 'Fantasy', 'Comedy'],
       dtype=object),
 'id': 502356,
 'imdb_id': 'tt6718170',
 'original_language': 'en',
 'original_title': 'The Super Mario Bros. Movie',
 'overview': 'While working underground to fix a water main, Brooklyn plumbers—and brothers—Mario and Luigi are transported down a mysterious pipe and wander into a magical new world. But when the brothers are separated, Mario embarks on an epic quest to find Luigi.',
 'popularity': 6572.614,
 'poster_path': 'https://image.tmdb.org/t/p/w500//qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg',
 'production_companies': array([{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKXoALWKdp0.png', 'name': 'Universal Pictures', 'origin_country': 'US'},
        {'id': 6704, 'logo_path': '/fOG2oY4m1YuYTQh4bMqqZkmgOAI.png', 'name': 'Illumination', 'origin_country': 'US'},
        {'id': 12288, 'logo_path

In [16]:
similarity_matrix = sim_matrix(df)

In [18]:
rec_list = process_reccom(df, movie['id'], similarity_matrix)
rec = get_top_recommended_movies(df, rec_list, 10)
rec[0]['points']

10.0

In [11]:
sim_dict = description_similarity_dic(502356, similarity_matrix, list(df['id']))
list_res=sacar_puntuaciones_completo(df,movie['keywords'],movie['genres'],movie['cast'],movie['director'],movie['year'], sim_dict)
list_res

[(640146,
  1.4412256847529294,
  {'keywords': 1.25,
   'genres': 1.4285714285714284,
   'cast': 0.0,
   'years': 10.0,
   'director': 0.0,
   'description': 0.836331310440289}),
 (502356,
  13.75,
  {'keywords': 10,
   'genres': 10,
   'cast': 10,
   'years': 10.0,
   'director': 10,
   'description': 25.0}),
 (594767,
  2.0799380106436614,
  {'keywords': 1.5384615384615385,
   'genres': 3.333333333333333,
   'cast': 0.0,
   'years': 10.0,
   'director': 0.0,
   'description': 1.1402648630874657}),
 (76600,
  0.9442793817105011,
  {'keywords': 0.0,
   'genres': 1.4285714285714284,
   'cast': 0.0,
   'years': 8.958341352965283,
   'director': 0.0,
   'description': 0.5568778276775197}),
 (948713,
  0.8240091661348162,
  {'keywords': 0.0,
   'genres': 1.1111111111111112,
   'cast': 0.0,
   'years': 10.0,
   'director': 0.0,
   'description': 0.18492555342815387}),
 (677179,
  0.654968087106147,
  {'keywords': 0.0,
   'genres': 0.0,
   'cast': 0.0,
   'years': 10.0,
   'director': 0.0,
 

In [11]:
# points_list = [] 
# for id, dic in dic_punt.items():
#     points_list.append((id, dic['points']))
# points_list.sort(key=lambda x: x[1], reverse=True)
# points_list

list_res.sort(key=lambda x: x[1], reverse=True)
list_res[:5]

[(502356,
  10.0,
  {'keywords_p': 3.0,
   'genres_p': 2.5,
   'cast_p': 0.5,
   'years_p': 0.5,
   'director_p': 1.0,
   'description_p': 2.5}),
 (585083,
  3.591258249850514,
  {'keywords_p': 0.49999999999999994,
   'genres_p': 2.5,
   'cast_p': 0.0,
   'years_p': 0.4479170676482641,
   'director_p': 0.0,
   'description_p': 0.14334118220224953}),
 (508439,
  3.433740635366663,
  {'keywords_p': 0.21428571428571425,
   'genres_p': 2.5,
   'cast_p': 0.02631578947368421,
   'years_p': 0.3594618667159631,
   'director_p': 0.0,
   'description_p': 0.3336772648913014}),
 (39691,
  3.330199398528727,
  {'keywords_p': 0.6666666666666666,
   'genres_p': 2.5,
   'cast_p': 0.0,
   'years_p': 0.11965446112187728,
   'director_p': 0.0,
   'description_p': 0.04387827074018289}),
 (315162,
  3.282567057511626,
  {'keywords_p': 0.23076923076923078,
   'genres_p': 2.5,
   'cast_p': 0.0,
   'years_p': 0.4479170676482641,
   'director_p': 0.0,
   'description_p': 0.103880759094131})]

In [68]:

filtered_results = [x[0] for x in list_res][:10]

filtered_df = df[df['id'].isin(filtered_results)]
filtered_df

,backdrop_path,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,title,vote_average,vote_count,cast,keywords,director,year,director_dict,cast_dict,description
8072,/iJQIbOPm81fPEGKt5BPuZmfnA54.jpg,100000000,"[Animation, Adventure, Family, Fantasy, Comedy]",502356,tt6718170,en,The Super Mario Bros. Movie,"While working underground to fix a water main,...",6572.614,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,...,The Super Mario Bros. Movie,7.515,1462,"[Jack Black, Chris Pratt, Anya Taylor-Joy, Cha...","[video gam, plumber, magic mushroom, based on ...","[Michael Jelenic, Aaron Horvath]",2023,"[{'adult': False, 'credit_id': '61260ba84e6742...","[{'adult': False, 'cast_id': 14, 'character': ...","While working underground to fix a water main,..."
8079,/ouB7hwclG7QI3INoYJHaZL4vOaa.jpg,90000000,"[Animation, Family, Fantasy, Adventure, Comedy...",315162,tt3915174,en,Puss in Boots: The Last Wish,Puss in Boots discovers that his passion for a...,1347.259,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg,...,Puss in Boots: The Last Wish,8.298,5332,"[Florence Pugh, Salma Hayek, John Mulaney, Ant...","[fairy tal, talking dog, spin off, aftercredit...",[Joel Crawford],2022,"[{'adult': False, 'credit_id': '6052483bd2147c...","[{'adult': False, 'cast_id': 30, 'character': ...",Puss in Boots discovers that his passion for a...
8095,/xVFpOIVdVn5ZUq1QIf0UJcZWjO2.jpg,12300000,"[Animation, Comedy, Family, Adventure, Fantasy]",816904,tt23177868,es,Momias,"Through a series of unfortunate events, three ...",749.770,/qVdrYN8qu7xUtsdEFeGiIVIaYd.jpg,...,Mummies,7.164,220,"[Sean Bean, Celia Imrie, Eleanor Tomlinson, Hu...","[egypt, museum, mummi, chariot rac, music, wed...",[Juan Jesús García Galocha],2023,"[{'adult': False, 'credit_id': '60719ac4cc277c...","[{'adult': False, 'cast_id': 39, 'character': ...","Through a series of unfortunate events, three ..."
8219,/7QabKu8tizoqy8qCZJXljdSpP4A.jpg,0,"[Animation, Comedy, Family, Fantasy, Adventure]",585083,tt9848626,en,Hotel Transylvania: Transformania,"When Van Helsing's mysterious invention, the ""...",179.967,/teCy1egGQa0y8ULJvlrDHQKnxBL.jpg,...,Hotel Transylvania: Transformania,7.053,1487,"[Andy Samberg, Selena Gomez, Kathryn Hahn, Ste...","[monster, vampir, transform, aftercreditssting...","[Derek Drymon, Jennifer Kluska]",2022,"[{'adult': False, 'credit_id': '602e9242d55c3d...","[{'adult': False, 'cast_id': 2, 'character': '...","When Van Helsing's mysterious invention, the ""..."
8409,/620hnMVLu6RSZW6a5rwO8gqpt0t.jpg,200000000,"[Animation, Comedy, Fantasy, Adventure, Family]",508943,tt12801262,en,Luca,Luca and his best friend Alberto experience an...,116.582,/8tABCBpzu3mZbzMB3sRzMEHEvJi.jpg,...,Luca,7.884,7239,"[Jack Dylan Grazer, Jacob Tremblay, Maya Rudol...","[itali, monster, friendship, friend, coming of...",[Enrico Casarosa],2021,"[{'adult': False, 'credit_id': '5f22cfa5350398...","[{'adult': False, 'cast_id': 10, 'character': ...",Luca and his best friend Alberto experience an...
8841,/q62bpQ67qaXY0u6b2wFEnQYIbPd.jpg,175000000,"[Family, Adventure, Animation, Comedy, Fantasy]",301528,tt1979376,en,Toy Story 4,Woody has always been confident about his plac...,71.492,/w9kR8qbmQ01HwnvK4alvnQ2ca0L.jpg,...,Toy Story 4,7.533,8780,"[Tom Hanks, Keanu Reeves, Annie Potts, Timothy...","[friendship, cartoon, sequel, rescue miss, bud...",[Josh Cooley],2019,"[{'adult': False, 'credit_id': '5c6ebb52925141...","[{'adult': False, 'cast_id': 6, 'character': '...",Woody has always been confident about his plac...
9023,/bNgqt819qpHcszjCzLCG5y16ldF.jpg,135000000,"[Animation, Adventure, Family, Fantasy, Comedy...",49519,tt0481499,en,The Croods,The prehistoric Croods family live in a partic...,57.877,/p7lJkqHlK01nr0zNacunUFI5Qxy.jpg,...,The Croods,6.909,6419,"[Nicolas Cage, Ryan Reynolds, Emma Stone, Cath...","[parent child relationship, stone ag, daughter...","[Chris Sanders, Kirk DeMicco]",2013,"[{'adult': False, 'credit_id': '52fe4799c3a368...","[{'adult': False, 'cast_id': 3, 'character': '...",T

In [ ]:
def compararString(str1, str2):
    if str1 == str2: return 10
    else: return 0

def compararListas(lista1, lista2):
    try:
        if len(lista1)== 0 or len(lista2) == 0:
            return 0
        set1 = set(lista1)
        set2 = set(lista2)
        if set1.issubset(set2):
            return 10
        else:
            intersection = set1.intersection(set2)
            union = set1.union(set2)
            jaccard_similarity = len(intersection) / len(union)
            score = jaccard_similarity * 10
        return score
    except:
        return 0


def similitud_anios(anio1, anio2, k):
    # Calcular la diferencia de años entre los dos valores
    diferencia = abs(anio1 - anio2)
    # Calcular la similitud utilizando la fórmula exponencial
    similitud = math.exp(-k * diferencia)
    return similitud
def sacarPuntuaciones(df, kw_list, genero, actores, director, año):
    dicResults = {}
    for index, row in df.iterrows():
        puntuacion = 0
        puntuacion += (compararListas(kw_list, row["keywords"]))*0.4
        puntuacion += (compararListas(genero, row["genres"]))*0.3 #(0.6)
        puntuacion += (compararListas(actores, row["cast"]))*0.1 #0.75
        puntuacion += (compararListas(director,row["director"]))*0.1 #0.8
        puntuacion += similitud_anios(int(año), int(row["year"]), 0.13)
        dicResults[row["id"]] = puntuacion
    return dicResults


In [27]:

movies = df[df['id'] == 502356]
for index, row in movies.iterrows():
        movie = row.to_dict()

movie

8072


{'backdrop_path': '/iJQIbOPm81fPEGKt5BPuZmfnA54.jpg',
 'budget': 100000000,
 'genres': array(['Animation', 'Adventure', 'Family', 'Fantasy', 'Comedy'],
       dtype=object),
 'id': 502356,
 'imdb_id': 'tt6718170',
 'original_language': 'en',
 'original_title': 'The Super Mario Bros. Movie',
 'overview': 'While working underground to fix a water main, Brooklyn plumbers—and brothers—Mario and Luigi are transported down a mysterious pipe and wander into a magical new world. But when the brothers are separated, Mario embarks on an epic quest to find Luigi.',
 'popularity': 6572.614,
 'poster_path': '/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg',
 'production_companies': array([{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKXoALWKdp0.png', 'name': 'Universal Pictures', 'origin_country': 'US'},
        {'id': 6704, 'logo_path': '/fOG2oY4m1YuYTQh4bMqqZkmgOAI.png', 'name': 'Illumination', 'origin_country': 'US'},
        {'id': 12288, 'logo_path': '/e4dQAqZD374H5EuM0W1ljEBWTKy.png', 'name': 'Nintendo', 'orig

In [6]:
dic_punt=sacarPuntuaciones(df,movie['keywords'],movie['genres'],movie['cast'],movie['director'],movie['year'])

In [10]:
# Ordenar el diccionario por valores
diccionario_ordenado = sorted(dic_punt.items(), key=lambda x: x[1], reverse=True)

# Iterar sobre el diccionario ordenado e imprimir cada clave y valor
for clave, valor in diccionario_ordenado:
    print(clave, valor)


ids = [x[0] for x in diccionario_ordenado][:10]

502356 10.0
585083 4.544762097587228
816904 4.2
315162 4.185787738612869
592867 4.165949119398766
39691 4.073408412881878
508439 4.015402739159819
676710 3.8780954309205615
676701 3.8780954309205615
778855 3.8780954309205615
560052 3.8780954309205615
301528 3.861187214636861
404368 3.8451226998379395
508943 3.7710515858035665
639721 3.7710515858035665
667276 3.7710515858035665
522478 3.7710515858035665
676705 3.7710515858035665
597316 3.7710515858035665
410113 3.7710515858035665
756403 3.7710515858035665
682254 3.7710515858035665
49519 3.7431200283281303
446893 3.729688453445533
529203 3.677056874498165
400160 3.677056874498165
137106 3.6032962341947776
330457 3.5945205479701943
400157 3.5945205479701943
508965 3.5945205479701943
615774 3.5945205479701943
458253 3.5945205479701943
462650 3.5695171224163347
19585 3.5256621145702445
370567 3.522045776761016
354279 3.522045776761016
438747 3.511037590252592
211672 3.4807274092315077
662708 3.4770568744981647
41513 3.4768027378674313
22816

In [12]:
filtered_points = df[df['id'].isin(ids)]
filtered_points

,backdrop_path,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,title,vote_average,vote_count,cast,keywords,director,year,director_dict,cast_dict,description
8072,/iJQIbOPm81fPEGKt5BPuZmfnA54.jpg,100000000,"[Animation, Adventure, Family, Fantasy, Comedy]",502356,tt6718170,en,The Super Mario Bros. Movie,"While working underground to fix a water main,...",6572.614,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,...,The Super Mario Bros. Movie,7.515,1462,"[Jack Black, Chris Pratt, Anya Taylor-Joy, Cha...","[video gam, plumber, magic mushroom, based on ...","[Michael Jelenic, Aaron Horvath]",2023,"[{'adult': False, 'credit_id': '61260ba84e6742...","[{'adult': False, 'cast_id': 14, 'character': ...","While working underground to fix a water main,..."
8079,/ouB7hwclG7QI3INoYJHaZL4vOaa.jpg,90000000,"[Animation, Family, Fantasy, Adventure, Comedy...",315162,tt3915174,en,Puss in Boots: The Last Wish,Puss in Boots discovers that his passion for a...,1347.259,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg,...,Puss in Boots: The Last Wish,8.298,5332,"[Florence Pugh, Salma Hayek, John Mulaney, Ant...","[fairy tal, talking dog, spin off, aftercredit...",[Joel Crawford],2022,"[{'adult': False, 'credit_id': '6052483bd2147c...","[{'adult': False, 'cast_id': 30, 'character': ...",Puss in Boots discovers that his passion for a...
8095,/xVFpOIVdVn5ZUq1QIf0UJcZWjO2.jpg,12300000,"[Animation, Comedy, Family, Adventure, Fantasy]",816904,tt23177868,es,Momias,"Through a series of unfortunate events, three ...",749.770,/qVdrYN8qu7xUtsdEFeGiIVIaYd.jpg,...,Mummies,7.164,220,"[Sean Bean, Celia Imrie, Eleanor Tomlinson, Hu...","[egypt, museum, mummi, chariot rac, music, wed...",[Juan Jesús García Galocha],2023,"[{'adult': False, 'credit_id': '60719ac4cc277c...","[{'adult': False, 'cast_id': 39, 'character': ...","Through a series of unfortunate events, three ..."
8106,/70aVSo3fuZ94jyQ3rT64afEf8lV.jpg,17000000,"[Animation, Adventure, Comedy, Family, Fantasy]",676710,tt10473036,en,The Amazing Maurice,Maurice is a streetwise ginger cat who comes u...,515.629,/ydhZeUjbzVEFclUpMhLfDZSavUY.jpg,...,The Amazing Maurice,7.033,61,"[Emilia Clarke, Gemma Arterton, David Thewlis,...","[based on novel or book, cat, grim reap, nine ...",[Toby Genkel],2022,"[{'adult': False, 'credit_id': '5e54674015959f...","[{'adult': False, 'cast_id': 12, 'character': ...",Maurice is a streetwise ginger cat who comes u...
8219,/7QabKu8tizoqy8qCZJXljdSpP4A.jpg,0,"[Animation, Comedy, Family, Fantasy, Adventure]",585083,tt9848626,en,Hotel Transylvania: Transformania,"When Van Helsing's mysterious invention, the ""...",179.967,/teCy1egGQa0y8ULJvlrDHQKnxBL.jpg,...,Hotel Transylvania: Transformania,7.053,1487,"[Andy Samberg, Selena Gomez, Kathryn Hahn, Ste...","[monster, vampir, transform, aftercreditssting...","[Derek Drymon, Jennifer Kluska]",2022,"[{'adult': False, 'credit_id': '602e9242d55c3d...","[{'adult': False, 'cast_id': 2, 'character': '...","When Van Helsing's mysterious invention, the ""..."
8338,/tUBN1paMQ1tmVA5Sjy1ZjPWVsiF.jpg,0,"[Animation, Adventure, Family, Comedy, Fantasy]",676701,tt14941698,es,Tadeo Jones 3: La Tabla Esmeralda,"Tad accidentally unleashes an ancient spell, e...",126.583,/jvIVl8zdNSOAJImw1elQEzxStMN.jpg,...,"Tad, the Lost Explorer and the Emerald Tablet",7.651,235,"[Michelle Jenner, Cecilia Suárez, Alexandra Ji...",None,[Enrique Gato],2022,"[{'adult': False, 'credit_id': '5e546009f48b34...","[{'adult': False, 'cast_id': 11, 'character': ...","Tad accidentally unleashes an ancient spell, e..."
9499,/7neeXze9HB3MTxDYyh3wwysydMN.jpg,20000000,"[Animation, Adventure, Comedy, Family, Fantasy]",778855,tt12532368,en,Chickenhare and the Hamster of Darkness,Young Chickenhare is the adopted son of King P...,40.603,/2v6f1EqTo4FUxVT9MDXpF6158AX.jpg,...,Chickenhare and the Hamster of Darkness,7.487,223,"[Dino Andrade, Jeff Doucette, Joe Ochman, Dann...",None,"[Ben Stassen, Benjamin Mousquet]",2022,"[{'adult': False, 'credit_id': '5fe28367e640d6...","[{'adult

# SISTEMA DE BÚSQUEDA POR TÍTULO Y DIRECTOR

In [ ]:
def buscar_titulo(cadena,df):
    pelis =  df[df['title'].str.contains(cadena, case=False)]
    return pelis['title']
#estaría bien que devolviera las pelis ordenadas de menos a mayor longitud de cadena, para ganar precisón

buscar_titulo('Batman',df)

In [ ]:
df['long_title'] = df['title'].str.len()
df.head()
def alternativa_buscar_tiulo(cadena,df):
    pelis =  df[df['title'].str.contains(cadena, case=False)]
    return df[pelis['title']].sort_values(by=['long_title'],ascending=True)
    
    
#estaría bien que devolviera las pelis ordenadas de menos a mayor longitud de cadena, para ganar precisón
#NO FUNCIONA

In [ ]:
from thefuzz import process

def search_titles(movie_title, titles_list, limit=5):
    matches = process.extract(movie_title, titles_list, limit=limit)
    matched_titles = [m[0] for m in matches]
    return matched_titles

# filtered_movies = df[df["title"].isin(matched_titles)]
# filtered_movies

print(search_titles('Batman', df['title'],limit=20))

In [ ]:
!pip install thefuzz[speedup]

In [ ]:
def buscar_director(director, df):
    
    df_exploded = df.explode('director')
    peliculas_director = df_exploded[df_exploded['director'].str.contains(director, case=False)]
    return peliculas_director['Titulo']

# problema porque al ser listas no se puede hacer como en title, hay que arreglarlo

In [3]:
buscar_director('Nolan',df)

NameError: name 'buscar_director' is not defined

# Pruebas

In [5]:
def sim_matrix(df):
    description_list = df["description"]
    # Crea una instancia del vectorizador TF-IDF
    tfidf = TfidfVectorizer()
    # Transforma las descripciones en una matriz de términos y documentos (DTM)
    dtm = tfidf.fit_transform(description_list)
    # Calcula la similitud coseno entre todas las películas
    similarity_matrix = cosine_similarity(dtm)
    # title_index_dic = {}
    # for index, id in enumerate(df['id']):
    #     title_index_dic[id] = index
    return similarity_matrix


def description_similarity_list(title_id, similarity_matrix, id_list):
    title_idx = id_list.index(title_id)
    title_row = list(similarity_matrix[title_idx])
    similarity_list = list(enumerate(title_row))
    similarity_list.sort(key= lambda x: x[1], reverse=True)
    return similarity_list

def description_similarity_dic(title_id, similarity_matrix, id_list):
    title_idx = id_list.index(title_id)
    title_row = list(similarity_matrix[title_idx])
    similarity_dict = dict(enumerate(title_row))
    return similarity_dict


# def get_description_recomenndations(
#     similarity_dic: dict[float:str], title_row, limit: int = 100
# ):
#     title_row.sort(reverse = True)
#     print(title_row)
#     recommendations = []
#     for similarity in title_row:
#         for id in similarity_dic[similarity]:
#             recommendations.append(id)


#     return recommendations

In [16]:
similarity_matrix = sim_matrix(df)
len(similarity_matrix)

9973

In [9]:
sim_list = description_similarity_list(640146, similarity_matrix, list(df['id']))
sim_list[:10]

[(0, 1.0000000000000002),
 (289, 0.4370464739996258),
 (347, 0.28706313256843563),
 (1265, 0.19331317561829608),
 (4424, 0.14561530805727016),
 (3756, 0.14513887579991963),
 (6627, 0.14252532043844734),
 (7430, 0.13413207104470634),
 (2551, 0.12787841196272376),
 (2950, 0.12500400540361106)]

In [8]:
lista = []
for i in range(1, 11):
    index = sim_list[i][0]
    id = df['id'][index]
    lista.append(id)

print(lista)
filtered = df[df['id'].isin(lista)]

movies = []
for index, row in filtered.iterrows():
    movie = row.to_dict()
    movie['tfidf'] = sim_list[index][1]
    movies.append(movie)

[x['title'] for x in movies]




[518527, 700348, 1069884, 13596, 480042, 11283, 1050167, 475, 700267, 1115545]


['Nanny McPhee',
 'Kangaroo Valley',
 'Bonnie and Clyde',
 'Escape Plan: The Extractors',
 'Untitled PVT04',
 'Wife for Rent',
 'Harry & Meghan: A Royal Romance',
 'The Intern - A Summer of Lust',
 'Travel Log',
 "My Best Friend's Girl"]

In [17]:
sim_dict = description_similarity_dic(640146, similarity_matrix, list(df['id']))
sim_dict

{0: 1.0000000000000002,
 1: 0.03345325241761156,
 2: 0.023396526161165374,
 3: 0.05257322014539292,
 4: 0.03340819169144504,
 5: 0.017526591559475314,
 6: 0.004882407953748874,
 7: 0.034935027876463486,
 8: 0.04878046466052789,
 9: 0.03720611436148295,
 10: 0.03889872871123213,
 11: 0.012281548973633087,
 12: 0.03815185895189984,
 13: 0.030858982220460757,
 14: 0.018391668415292755,
 15: 0.01813808430133835,
 16: 0.013759479062001402,
 17: 0.029160809044718773,
 18: 0.0444167456825256,
 19: 0.020525265862393862,
 20: 0.04716405618513535,
 21: 0.04222264831094415,
 22: 0.03892733051628836,
 23: 0.0449886737458038,
 24: 0.03955800853476708,
 25: 0.033552906100011784,
 26: 0.026151826449511625,
 27: 0.03887676379678642,
 28: 0.012526754876373647,
 29: 0.054185365373530234,
 30: 0.027861788474214253,
 31: 0.03609154303307095,
 32: 0.0134673429055443,
 33: 0.05067908732568125,
 34: 0.033763525361409745,
 35: 0.03586812169049293,
 36: 0.039589243485337475,
 37: 0.016403103120038603,
 38: 0.0

In [11]:

def process_similarity(df, title_id):
    similarity_matrix = sim_matrix(df)
    sim_list = description_similarity(title_id, similarity_matrix, list(df["id"]))
    lista = []
    for i in range(1, 1000):
        index = sim_list[i][0]
        id = df["id"][index]
        lista.append(id)

    filtered = df[df["id"].isin(lista)]
    filtered["tfidf"] = [sim_list[i][1] for i in range(1000)]
    return filtered

process_similarity(df, 640146)

KeyError: 7647

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def compararString(str1, str2):
    if str1 == str2:
        return 10
    else:
        return 0


def compararListas(lista1, lista2):
    try:
        if len(lista1) == 0 or len(lista2) == 0:
            return 0
        set1 = set(lista1)
        set2 = set(lista2)
        if set1.issubset(set2):
            return 10
        else:
            intersection = set1.intersection(set2)
            union = set1.union(set2)
            jaccard_similarity = len(intersection) / len(union)
            score = jaccard_similarity * 10
        return score
    except:
        return 0


def similitud_anios(anio1, anio2, k):
    # Calcular la diferencia de años entre los dos valores
    diferencia = abs(anio1 - anio2)
    # Calcular la similitud utilizando la fórmula exponencial
    similitud = math.exp(-k * diferencia)
    return similitud


def sim_matrix(df):
    description_list = df["description"]
    # Crea una instancia del vectorizador TF-IDF
    tfidf = TfidfVectorizer()
    # Transforma las descripciones en una matriz de términos y documentos (DTM)
    dtm = tfidf.fit_transform(description_list)
    # Calcula la similitud coseno entre todas las películas
    similarity_matrix = cosine_similarity(dtm)
    # title_index_dic = {}
    # for index, id in enumerate(df['id']):
    #     title_index_dic[id] = index
    return similarity_matrix


def description_similarity_dic(title_id, similarity_matrix, id_list):
    title_idx = id_list.index(title_id)
    title_row = list(similarity_matrix[title_idx])
    similarity_dict = dict(enumerate(title_row))
    return similarity_dict


def sacar_puntuaciones_completo(df, kw_list, genero, actores, director, año, sim_dict):
    results_list = []
    i = 0
    for index, row in df.iterrows():
        keywords_p = compararListas(kw_list, row["keywords"])  # Sobre 10
        genres_p = compararListas(genero, row["genres"])  # Sobre 10
        cast_p = compararListas(actores, row["cast"])  # Sobre 10
        years_p = similitud_anios(int(año), int(row["year"]), 0.11)  # Sobre 1
        director_p = compararListas(director, row["director"])  # Sobre 10
        description_p = sim_dict[i]  # sobre 1
        # kw * 0.35; genres * 0.15; cast * 0.05; years * 0.5; director * 0.1; description * 3

        labels = [keywords_p, genres_p, cast_p, years_p, director_p, description_p]
        values = [0.35, 0.15, 0.05, 0.5, 0.1, 3]
        points = sum(list(map(lambda x, y: x * y, labels, values)))
        results = {
            "keywords": keywords_p,
            "genres": genres_p,
            "cast": cast_p,
            "years": years_p * 10,
            "director": director_p,
            "description": description_p * 10,
        }
        results_list.append((row["id"], points, results))
        i += 1
    return results_list


def process_reccom(df, movie_id, similarity_matrix):
    # sacar_puntuaciones_completo(df, kw_list, genero, actores, director, año, sim_dict)
    movies = df[df["id"] == movie_id]
    for index, row in movies.iterrows():
        movie = row.to_dict()

    sim_dict = description_similarity_dic(movie_id, similarity_matrix, list(df["id"]))
    list_res = sacar_puntuaciones_completo(
        df,
        movie["keywords"],
        movie["genres"],
        movie["cast"],
        movie["director"],
        movie["year"],
        sim_dict,
    )
    list_res.sort(key=lambda x: x[1], reverse=True)
    return list_res


def get_top_recommended_movies(
    df: pd.DataFrame, recommendations_list: list, limit: int
):
    def search_points(rec_list: list, id: int, limit: int):
        for i in range(1, limit + 1):
            m = rec_list[i]
            if m[0] == id:
                return (recommendations_list[i][1], recommendations_list[i][2])
        return None, None

    filtered_results = [x[0] for x in recommendations_list][1 : limit + 1]
    filtered_df = df[df["id"].isin(filtered_results)]
    movies = []
    for idx, row in filtered_df.iterrows():
        movie = row.to_dict()
        points, results = search_points(recommendations_list, movie["id"], limit)
        movie["points"] = points
        movie["results"] = results
        movies.append(movie)
    return movies

In [1]:
from python_files.tmdb import TMDB_wrapper
tmdb = TMDB_wrapper()

In [6]:
movie_id = 505642
recommendations = tmdb.get_recommendations(movie_id)
ids = [x["id"] for x in recommendations]
filtered = df[df["id"].isin(ids)]
movies = []
for index, row in filtered.iterrows():
    movies.append(row.to_dict())
print(len(recommendations), len(ids), len(movies))

[{'adult': False,
  'backdrop_path': '/9Rq14Eyrf7Tu1xk0Pl7VcNbNh1n.jpg',
  'id': 646389,
  'title': 'Plane',
  'original_language': 'en',
  'original_title': 'Plane',
  'overview': 'After a heroic job of successfully landing his storm-damaged aircraft in a war zone, a fearless pilot finds himself between the agendas of multiple militias planning to take the plane and its passengers hostage.',
  'poster_path': '/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg',
  'media_type': 'movie',
  'genre_ids': [28, 12, 53],
  'popularity': 370.387,
  'release_date': '2023-01-12',
  'video': False,
  'vote_average': 6.872,
  'vote_count': 1192},
 {'adult': False,
  'backdrop_path': '/aHcWyh0n4YbBy9gxYCDlgsNVS3M.jpg',
  'id': 631842,
  'title': 'Knock at the Cabin',
  'original_language': 'en',
  'original_title': 'Knock at the Cabin',
  'overview': 'While vacationing at a remote cabin, a young girl and her two fathers are taken hostage by four armed strangers who demand that the family make an unthinkable choice 